<a href="https://colab.research.google.com/github/LukrecijaTudor/NLP/blob/main/DistilBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pycld2

     |████████████████████████████████| 41.4MB 77kB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp36-cp36m-linux_x86_64.whl size=9833532 sha256=2b6b0df135042f171bd1fc3242453a417724366837e453feac3a38ee3ff4c385
  Stored in directory: /root/.cache/pip/wheels/c6/8f/e9/08a1a8932a490175bd140206cd86a3dbcfc70498100de11079
Successfully built pycld2


In [ ]:
pip install transformers

     |████████████████████████████████| 1.8MB 17.2MB/s 
     |████████████████████████████████| 2.9MB 56.0MB/s 
     |████████████████████████████████| 890kB 55.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=ba4417dfcdaa8b7b8d24d833be97273f17aaa72c4cff28e1e3aebd3b7dacc840
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import pandas as pd
import pycld2 as cld2
#import ndjson
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
#baseline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [ ]:
acronyms = pd.read_excel("/content/kratice.xlsx")
acronyms = acronyms.rename(columns={'kratice':'acronym','Unnamed: 1':'replacement'})
document = pd.read_csv("/content/qa06_all.csv")
document = document.rename(columns={'qa06id': 'id', 'qa06name':'title', 'qa06wher':'location','qa06dsc':'report'})
document_risk=pd.read_csv("/content/qa06_only_having_risk_valuesl.csv")
document_risk = document_risk.rename(columns={'qa06id': 'id', 'qa06name':'title', 'qa06wher':'location','qa06dsc':'report','ty26colo':'label','ty26fakt':'factor'})
def acronyms_to_words (txt):
    #acronyms = pd.read_excel("/Users/Lukre/Desktop/D/kratice.xlsx")
    #acronyms = acronyms.rename(columns={'kratice':'acronym','Unnamed: 1':'replacement'})
    for i in range (len(acronyms)):
        txt=txt.replace(' ' + acronyms.acronym[i] + ' ',' ' + acronyms.replacement[i] + ' ')
    return txt

def isNaN(string):
    return string != string
def prep_doc__for_bert_classification(document):
    
  doc = pd.DataFrame(columns=['report','accident','label'])
    
  for i in range(len(document)):
       
      if isNaN(document.report[i]):
          continue
      isReliable, textBytesFound, details=cld2.detect(document.report[i])
      
      if not isReliable:
                  continue
      if details[0][0]!='ENGLISH':
          continue
      txt=acronyms_to_words (document.report[i])
      if (document.factor[i]==1):
                pom_2=0
                pom='no accident outcome'
      
      if (document.factor[i] in (2,4,20,100)):
                pom_2=1
                pom='minor injuries or damage'
      if (document.factor[i] in (10,50,21,102,101,502,500)):
                pom_2=2
                pom='major or catastrophic accident'

      doc=doc.append({'report': txt,'accident': pom,'label': pom_2},ignore_index=True)

  return doc
def train_val_test_split_ (doc,label,rnd=42,size=0.2):
  X_train_1, X_test, y_train_1, y_test = train_test_split(doc.index.values, 
                                                  doc.label.values, 
                                                  test_size=size, 
                                                  random_state=rnd, 
                                                  stratify=doc.label.values)
  
  pom = pd.DataFrame(columns=['X','y'])
  pom['X']=X_train_1
  pom['y']=y_train_1

  X_train, X_val, y_train, y_val = train_test_split(pom.X.values, 
                                                  pom.y.values, 
                                                  test_size=size, 
                                                  random_state=rnd, 
                                                  stratify=pom.y.values) 
  doc['data_type'] = ['not_set']*doc.shape[0]


  doc.loc[X_train, 'data_type'] = 'train'
  doc.loc[X_val, 'data_type'] = 'val' 
  doc.loc[X_test, 'data_type'] = 'test'
  return doc

In [ ]:
doc_bert = prep_doc__for_bert_classification (document_risk)

In [ ]:
texts=doc_bert['report'].to_list()
labels=doc_bert['accident'].to_list()

In [ ]:
main_txt, test_txt, main_labels, test_labels = train_test_split(texts, labels, test_size=0.1, random_state=1)
train_txt, val_txt, train_labels, val_labels = train_test_split(main_texts, rest_labels, test_size=0.1, random_state=1)

print("Train size:", len(train_txt))
print("Val size:", len(val_txt))
print("Test size:", len(test_txt))

In [ ]:
class_names = list(set(labels))
label_ids = {label: id for id, label in enumerate(class_names)}
print(label_ids)

{'minor injuries or damage': 0, 'no accident outcome': 1, 'major or catastrophic accident': 2}


MODEL

In [ ]:
from transformers import DistilBertTokenizer

BERT_MODEL = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(BERT_MODEL)

In [ ]:
from transformers import DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained(BERT_MODEL, num_labels = len(label_ids))
model.to(device)

In [ ]:
import logging

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

inputi modela

In [ ]:
MAX_SEQ_LENGTH = 128 #100, 128, 256, 512
BATCH_SIZE = 16  #16, 32

In [ ]:
class BertInput(object):

    def __init__(self, text, input_ids, input_mask, label_id):
        self.text = text
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.label_id = label_id
        

def input_data(example_texts, example_labels, label_ids, max_seq_length, tokenizer, verbose=0):
    
    input_items = []
    examples = zip(example_texts, example_labels)
    for (ex_index, (text, label)) in enumerate(examples):

        input_ids = tokenizer.encode(f"[CLS] {text} [SEP]")
        if len(input_ids) > max_seq_length:
            input_ids = input_ids[:max_seq_length]

        #segment_ids = [0] * len(input_ids)  --kod BERT modela se uvodi i svi se postave na vrijednost 1

        input_mask = [1] * len(input_ids)

        padding = [0] * (max_seq_length - len(input_ids)) # truncate i padding također su moguće opcije kod tokenizatora
        input_ids += padding
        input_mask += padding
        #segment_ids += padding --kod BERT modela

        #provjera valjanosti
        assert len(input_ids) == max_seq_length 
        assert len(input_mask) == max_seq_length
        #assert len(segment_ids) == max_seq_length

        label_id = label_ids[label]

        input_items.append(
            BertInput(text=text,
                          input_ids=input_ids,
                          input_mask=input_mask,
                          label_id=label_id))

        
    return input_items

In [ ]:
train_features = input_data(train_txt, train_labels, label_ids, MAX_SEQ_LENGTH, tokenizer, verbose=0)
val_features = input_data(val_txt, val_labels, label_ids, MAX_SEQ_LENGTH, tokenizer)
test_features = input_data(test_txt, test_labels, label_ids, MAX_SEQ_LENGTH, tokenizer)

Token indices sequence length is longer than the specified maximum sequence length for this model (513 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
import pickle

In [ ]:
  with open ('train_input_distilbert.csv','wb') as f:
    pickle.dump(train_features,f)
  with open ('val_input_distilbert.csv','wb') as g:
    pickle.dump(dev_features,g)
  with open ('test_input_distilbert.csv','wb') as h:
    pickle.dump(test_features,h)
  with open ('train_input_distilbert.csv','rb') as i:
    d=pickle.load(i)
  with open ('val_input_distilbert.csv','rb') as j:
    e=pickle.load(j)
  with open ('test_input_distilbert.csv','rb') as k:
    l=pickle.load(k)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler

def get_data_loader(features, max_seq_length, batch_size, shuffle=True): 

    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    #all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)
    data = TensorDataset(all_input_ids, all_input_mask, all_label_ids)

    dataloader = DataLoader(data, shuffle=shuffle, batch_size=batch_size)
    return dataloader

train_dataloader = get_data_loader(train_features, MAX_SEQ_LENGTH, BATCH_SIZE, shuffle=True)
val_dataloader = get_data_loader(val_features, MAX_SEQ_LENGTH, BATCH_SIZE, shuffle=False)
test_dataloader = get_data_loader(test_features, MAX_SEQ_LENGTH, BATCH_SIZE, shuffle=False)

In [ ]:
def evaluation(model, dataloader):
    model.eval()
    
    eval_loss = 0
    nb_eval_steps = 0
    predicted_labels, correct_labels = [], []

    for step, batch in enumerate(tqdm(dataloader, desc="Evaluation iteration")):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, label_ids = batch

        with torch.no_grad():
            tmp_eval_loss, logits = model(input_ids, attention_mask=input_mask, labels=label_ids,return_dict=False)

        outputs = np.argmax(logits.to('cpu'), axis=1)
        label_ids = label_ids.to('cpu').numpy()

        predicted_labels += list(outputs)
        correct_labels += list(label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    
    correct_labels = np.array(correct_labels)
    predicted_labels = np.array(predicted_labels)
        
    return eval_loss, correct_labels, predicted_labels

In [ ]:
from transformers.optimization import AdamW
from transformers.optimization import get_linear_schedule_with_warmup

GRADIENT_ACCUMULATION_STEPS = 1
NUM_TRAIN_EPOCHS = 5 #10 #20
LEARNING_RATE = 1e-5 #5e-5
WARMUP_PROPORTION = 0.1
MAX_GRAD_NORM = 5

num_train_steps = int(len(train_dataloader.dataset) / BATCH_SIZE / GRADIENT_ACCUMULATION_STEPS * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(WARMUP_PROPORTION * num_train_steps)

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

optimizer = AdamW(optimizer_grouped_parameters, lr=LEARNING_RATE, correct_bias=False)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_train_steps)


In [ ]:
import torch
import os
from tqdm import trange
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import classification_report, precision_recall_fscore_support

In [ ]:
loss_history = []
no_improvement = 0
for _ in trange(int(NUM_TRAIN_EPOCHS), desc="Epoch"):
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(tqdm(train_dataloader, desc="Training iteration")):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, label_ids = batch

        outputs = model(input_ids, attention_mask=input_mask, labels=label_ids, return_dict=False)
        loss = outputs[0]

        if GRADIENT_ACCUMULATION_STEPS > 1:
            loss = loss / GRADIENT_ACCUMULATION_STEPS

        loss.backward()
        tr_loss += loss.item()

        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)  
            
            optimizer.step()
            optimizer.zero_grad()
            scheduler.step()
            
    val_loss, _, _ = evaluation(model, val_dataloader)
    
    print("Loss history:", loss_history)
    print("Val loss:", dev_loss)
    
    if len(loss_history) == 0 or val_loss < min(loss_history):
        no_improvement = 0
        model_to_save = model.module if hasattr(model, 'module') else model

    else:
        no_improvement += 1

   loss_history.append(val_loss)

In [ ]:
model = model_to_save
model.to(device)

model.eval()

_, train_correct, train_predicted = evaluation(model, train_dataloader)
_, val_correct, val_predicted = evaluation(model, val_dataloader)
_, test_correct, test_predicted = evaluation(model, test_dataloader)

print("Training performance:", precision_recall_fscore_support(train_correct, train_predicted, average="micro"))
print("Development performance:", precision_recall_fscore_support(val_correct, val_predicted, average="micro"))
print("Test performance:", precision_recall_fscore_support(test_correct, test_predicted, average="micro"))

bert_accuracy = np.mean(test_predicted == test_correct)

print(classification_report(test_correct, test_predicted, target_names=target_names))

In [ ]:
import sklearn
y_test = test_correct
y_pred = test_predicted

In [ ]:
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_test, y_pred)
print('Confusion Matrix\n')
print(confusion)

#importing accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, y_pred)))

print('Micro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='micro')))

print('Macro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='macro')))
print('Macro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='macro')))
print('Macro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='macro')))

print('Weighted Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_test, y_pred, average='weighted')))